This notebook contains code for testing other machine learning models.

In [1]:
import pandas as pd
from datetime import datetime

import os
import json

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

cwd = os.getcwd()
with open(f'city_config.json') as f:
        city_data = json.load(f)

In [2]:
def combine_weather_data(city_dict, path, date):
    recent_nws = pd.read_csv(f'{path}/weather_forecast/{city_dict['issue']}/{date}/recent_NWS.csv',
                                index_col=0)

    historical_om = pd.read_csv(f'{path}/weather_forecast/{city_dict['issue']}/{date}/historical_openmeteo.csv',
                                index_col=0)
    
    historical_ms = pd.read_csv(f'{path}/weather_forecast/{city_dict['issue']}/{date}/historical_meteostat.csv',
                                index_col=0)

    forecast_aw = pd.read_csv(f'{path}/weather_forecast/{city_dict['issue']}/{date}/forecast_accuweather.csv',
                                index_col=0)
    
    forecast_nws = pd.read_csv(f'{path}/weather_forecast/{city_dict['issue']}/{date}/forecast_NWS.csv',
                                index_col=0)

    forecast_vc = pd.read_csv(f'{path}/weather_forecast/{city_dict['issue']}/{date}/forecast_visualcrossing.csv',
                                index_col=0)
    
    all_pandas = [recent_nws, historical_om, historical_ms, forecast_aw, forecast_nws, forecast_vc]
    
    combined_df = pd.concat(all_pandas, ignore_index=True)

    combined_df['Date'] = pd.to_datetime(combined_df['Date'])

    # Group by 'Date' and calculate the average temperature
    average_temperatures = combined_df.groupby('Date')['Temp'].mean().reset_index()
    average_temperatures['Temp'] = average_temperatures['Temp'].round(2)

    # Create separate columns for year, month, and date.
    average_temperatures['Year'] = pd.to_datetime(average_temperatures['Date']).dt.year
    average_temperatures['Month'] = pd.to_datetime(average_temperatures['Date']).dt.month
    average_temperatures['Day'] = pd.to_datetime(average_temperatures['Date']).dt.day

    return average_temperatures

### RNN

In [5]:
date = '2024.03.25'
cwd = os.getcwd()
for i in city_data.keys():
    print(i)
    # Load the data and preprocess
    data = combine_weather_data(city_data[i], cwd, date)
    data['Date'] = pd.to_datetime(data['Date'])
    data['Year'] = pd.to_datetime(data['Date']).dt.year
    data['Month'] = pd.to_datetime(data['Date']).dt.month
    data['Day'] = pd.to_datetime(data['Date']).dt.day

    # Extract features and normalize
    X = data[['Year', 'Month', 'Day']].values
    y = data['Temp'].values

    X_max, X_min = X.max(axis=0), X.min(axis=0)
    y_max, y_min = y.max(), y.min()

    X = (X - X_min) / (X_max - X_min)
    y = (y - y_min) / (y_max - y_min)

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Convert data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

    # Define the RNN model
    class SimpleRNN(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            super(SimpleRNN, self).__init__()
            self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, output_size)

        def forward(self, x):
            out, _ = self.rnn(x)
            out = self.fc(out[:, -1, :])  # Use the last output of the sequence
            return out

    # Define hyperparameters
    input_size = X_train.shape[1]  # Number of features (Year, Month, Day)
    hidden_size = 64  # Number of hidden units in the RNN
    output_size = 1  # Output size (temperature)

    # Initialize the model
    model = SimpleRNN(input_size, hidden_size, output_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()  # Mean Squared Error loss
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    epochs = 100
    for epoch in range(epochs):
        # Forward pass
        outputs = model(X_train_tensor.unsqueeze(1))  # Add an extra dimension for batch (batch_size=1)
        loss = criterion(outputs.squeeze(), y_train_tensor)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print progress
        if (epoch+1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

    # Evaluate the model
    with torch.no_grad():
        test_outputs = model(X_test_tensor.unsqueeze(1))  # Add an extra dimension for batch (batch_size=1)
        test_loss = criterion(test_outputs.squeeze(), y_test_tensor)
        print(f"Test Loss: {test_loss.item()}")

    # Make prediction for today
    today_features = (pd.to_datetime(date).year, pd.to_datetime(date).month, pd.to_datetime(date).day)
    today_normalized = (today_features - X_min) / (X_max - X_min)
    date_to_predict = torch.tensor([today_normalized], dtype=torch.float32).unsqueeze(1)  # Add an extra dimension for batch (batch_size=1)
    predicted_temperature = model(date_to_predict)
    predicted_temperature = predicted_temperature.item() * (y_max - y_min) + y_min  # Inverse normalization
    print(f"Predicted temperature for {date}:", predicted_temperature)

New York
Epoch 10/100, Loss: 0.28192591667175293
Epoch 20/100, Loss: 0.12199616432189941
Epoch 30/100, Loss: 0.057367272675037384
Epoch 40/100, Loss: 0.052057720720767975
Epoch 50/100, Loss: 0.05319874361157417
Epoch 60/100, Loss: 0.04972388222813606
Epoch 70/100, Loss: 0.04845818132162094
Epoch 80/100, Loss: 0.04791993275284767
Epoch 90/100, Loss: 0.047172464430332184
Epoch 100/100, Loss: 0.04657076671719551
Test Loss: 0.043743230402469635
Predicted temperature for 2024.03.25: 66.19944322705268
Austin
Epoch 10/100, Loss: 0.2079453319311142
Epoch 20/100, Loss: 0.07625186443328857
Epoch 30/100, Loss: 0.03475801274180412
Epoch 40/100, Loss: 0.03683088719844818


/var/folders/tz/30nwls9j5xn5nv008g9ngw4w0000gn/T/ipykernel_96636/1576359970.py:80: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  date_to_predict = torch.tensor([today_normalized], dtype=torch.float32).unsqueeze(1)  # Add an extra dimension for batch (batch_size=1)


Epoch 50/100, Loss: 0.035491906106472015
Epoch 60/100, Loss: 0.032716505229473114
Epoch 70/100, Loss: 0.03240877017378807
Epoch 80/100, Loss: 0.03194981813430786
Epoch 90/100, Loss: 0.031465452164411545
Epoch 100/100, Loss: 0.0311155766248703
Test Loss: 0.03160594403743744
Predicted temperature for 2024.03.25: 84.39489486455918
Chicago
Epoch 10/100, Loss: 0.27680110931396484
Epoch 20/100, Loss: 0.13451610505580902
Epoch 30/100, Loss: 0.06308562308549881
Epoch 40/100, Loss: 0.046067144721746445
Epoch 50/100, Loss: 0.047643646597862244
Epoch 60/100, Loss: 0.045708537101745605
Epoch 70/100, Loss: 0.04385094344615936
Epoch 80/100, Loss: 0.0432555228471756
Epoch 90/100, Loss: 0.04259509593248367
Epoch 100/100, Loss: 0.04194939509034157
Test Loss: 0.03897223621606827
Predicted temperature for 2024.03.25: 66.79134700775147
Miami
Epoch 10/100, Loss: 0.42610493302345276
Epoch 20/100, Loss: 0.2279249131679535
Epoch 30/100, Loss: 0.09663362056016922
Epoch 40/100, Loss: 0.034204740077257156
Epoch 

### Linear Regression

In [6]:
from sklearn.linear_model import LinearRegression

date = '2024.03.25'
cwd = os.getcwd()
for i in city_data.keys():
    print(i)
    # Load the data and preprocess
    data = combine_weather_data(city_data[i], cwd, date)
    data['Date'] = pd.to_datetime(data['Date'])
    data['Year'] = pd.to_datetime(data['Date']).dt.year
    data['Month'] = pd.to_datetime(data['Date']).dt.month
    data['Day'] = pd.to_datetime(data['Date']).dt.day

    # Extract features and normalize
    X = data[['Year', 'Month', 'Day']].values
    y = data['Temp'].values

    X_max, X_min = X.max(axis=0), X.min(axis=0)
    y_max, y_min = y.max(), y.min()

    X = (X - X_min) / (X_max - X_min)
    y = (y - y_min) / (y_max - y_min)

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the linear regression model
    model = LinearRegression()

    # Train the model
    model.fit(X_train, y_train)

    # Evaluate the model
    train_mse = mean_squared_error(y_train, model.predict(X_train))
    test_mse = mean_squared_error(y_test, model.predict(X_test))
    print("Train MSE:", train_mse)
    print("Test MSE:", test_mse)

    # Make prediction for today
    today_features = (pd.to_datetime(date).year, pd.to_datetime(date).month, pd.to_datetime(date).day)
    today_normalized = (today_features - X_min) / (X_max - X_min)
    date_to_predict = np.array([today_normalized])  # No need to unsqueeze for linear regression
    predicted_temperature = model.predict(date_to_predict.reshape(1, -1))
    predicted_temperature = predicted_temperature.item() * (y_max - y_min) + y_min  # Inverse normalization
    print(f"Predicted temperature for {date}:", predicted_temperature)

New York
Train MSE: 0.04196169228180865
Test MSE: 0.041097906570355586
Predicted temperature for 2024.03.25: 55.580445587898474
Austin
Train MSE: 0.02758916297371686
Test MSE: 0.028831846284349563
Predicted temperature for 2024.03.25: 75.51799105725168
Chicago
Train MSE: 0.037326517027466956
Test MSE: 0.03492970089327237
Predicted temperature for 2024.03.25: 53.67548612748775
Miami
Train MSE: 0.015805656527647042
Test MSE: 0.01460955729393199
Predicted temperature for 2024.03.25: 83.26223842295929


### Transformer

In [17]:
date = '2024.03.25'
cwd = os.getcwd()
for i in city_data.keys():
    print(i)
    # Load the data and preprocess
    data = combine_weather_data(city_data[i], cwd, date)
    data['Date'] = pd.to_datetime(data['Date'])
    data['Year'] = pd.to_datetime(data['Date']).dt.year
    data['Month'] = pd.to_datetime(data['Date']).dt.month
    data['Day'] = pd.to_datetime(data['Date']).dt.day

    # Extract features and normalize
    X = data[['Year', 'Month', 'Day']]
    y = data['Temp'].values

    # Convert to PyTorch tensors
    X_tensor = torch.from_numpy(X.values.astype(np.float32))
    y_tensor = torch.from_numpy(y.astype(np.float32))

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

    # Define the Transformer model
    class TransformerModel(nn.Module):
        def __init__(self, input_size, output_size, num_layers=2, num_heads=8, hidden_dim=64):
            super(TransformerModel, self).__init__()
            self.embedding = nn.Linear(input_size, hidden_dim)
            encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, batch_first=True)
            self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
            self.fc = nn.Linear(hidden_dim, output_size)

        def forward(self, x):
            x = self.embedding(x)
            x = x.unsqueeze(1)  # Add sequence length dimension
            output = self.transformer_encoder(x)
            output = self.fc(output[:, -1, :])  # Take the last time step and apply the linear layer
            return output

    # Example usage (assuming X_train, X_test, y_train, y_test are defined)
    input_size = X_train.shape[1]
    output_size = 1
    learning_rate = 0.001
    num_epochs = 500

    # Initialize the Transformer model
    model = TransformerModel(input_size, output_size)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training the model
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)  # Squeeze to remove extra dimension
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        test_loss = criterion(test_outputs.squeeze(), y_test)  # Squeeze to remove extra dimension
        print(f'Test Loss: {test_loss.item():.4f}')

    # Make prediction for 2024-03-25
    date_to_predict = torch.tensor([[2024, 3, 25]], dtype=torch.float32)
    predicted_temperature = model(date_to_predict)
    print(f"Predicted temperature for 2024-03-25:", predicted_temperature.item())

New York
Epoch [100/500], Loss: 2943.4688
Epoch [200/500], Loss: 1792.2217
Epoch [300/500], Loss: 889.2105
Epoch [400/500], Loss: 439.8386
Epoch [500/500], Loss: 309.8476
Test Loss: 307.2342
Predicted temperature for 2024-03-25: 59.134761810302734
Austin
Epoch [100/500], Loss: 4957.5664
Epoch [200/500], Loss: 3306.7136
Epoch [300/500], Loss: 1809.4319
Epoch [400/500], Loss: 827.5840
Epoch [500/500], Loss: 382.0824
Test Loss: 390.9588
Predicted temperature for 2024-03-25: 68.96963500976562
Chicago
Epoch [100/500], Loss: 2897.9297
Epoch [200/500], Loss: 1790.1531
Epoch [300/500], Loss: 946.4073
Epoch [400/500], Loss: 539.7171
Epoch [500/500], Loss: 428.5333
Test Loss: 419.6447
Predicted temperature for 2024-03-25: 57.632118225097656
Miami
Epoch [100/500], Loss: 5415.6211
Epoch [200/500], Loss: 3675.0730
Epoch [300/500], Loss: 2015.3003
Epoch [400/500], Loss: 852.1341
Epoch [500/500], Loss: 276.4267
Test Loss: 267.4150
Predicted temperature for 2024-03-25: 70.50745391845703


### Random Forest
#### Testing removing temperature of predicted date from dataset

In [16]:
# Load the data
date = '2024.03.25'
cwd = os.getcwd()
for i in city_data.keys():
    print(i)
    # Load the data and preprocess
    data = combine_weather_data(city_data[i], cwd, date)
    data['Date'] = pd.to_datetime(data['Date'])
    
    # Extract features
    X = data['Date'].dt.dayofyear.values.reshape(-1, 1)  # Convert date to day of the year
    y = data['Temp'].values

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the Random Forest Regressor
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    test_predictions = model.predict(X_test)

    # Compute and print the Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, test_predictions)
    print("Test MSE:", mse)

    # Make prediction for 2024-03-12
    date_to_predict = pd.to_datetime("2024-03-25").dayofyear
    predicted_temperature = model.predict([[date_to_predict]])
    print(f"Predicted temperature for {date}:", predicted_temperature[0])

    print(f'Removing forecast for {date}:')

    # Convert the "Date" column to datetime format
    # data['Date'] = pd.to_datetime(data['Date'])
    data = data[data['Date'].dt.date != pd.to_datetime(date).date()] #

    # Extract features
    X = data['Date'].dt.dayofyear.values.reshape(-1, 1)  # Convert date to day of the year
    y = data['Temp'].values

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the Random Forest Regressor
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    test_predictions = model.predict(X_test)

    # Compute and print the Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, test_predictions)
    print("Test MSE:", mse)

    # Make prediction for 2024-03-12
    date_to_predict = pd.to_datetime(date).dayofyear
    predicted_temperature = model.predict([[date_to_predict]])
    print(f"Predicted temperature for {date}:", predicted_temperature[0])
    print()



New York
Test MSE: 81.04347638516414
Predicted temperature for 2024.03.25: 59.99585390331886
Removing forecast for 2024.03.25:
Test MSE: 76.9017216351906
Predicted temperature for 2024.03.25: 62.282097142857175

Austin
Test MSE: 107.2199155810471
Predicted temperature for 2024.03.25: 80.01922142135642
Removing forecast for 2024.03.25:
Test MSE: 100.28662064792013
Predicted temperature for 2024.03.25: 81.14616071428573

Chicago
Test MSE: 122.01444689918206
Predicted temperature for 2024.03.25: 48.34487808802311
Removing forecast for 2024.03.25:
Test MSE: 123.26796779683092
Predicted temperature for 2024.03.25: 43.69466214285714

Miami
Test MSE: 18.281562827637753
Predicted temperature for 2024.03.25: 83.12555977633478
Removing forecast for 2024.03.25:
Test MSE: 20.113501729857088
Predicted temperature for 2024.03.25: 84.42585142857139



### Gradient Boosting
#### Testing removing temperature of predicted date from dataset

In [25]:
# Load the data
date = '2024.03.25'
cwd = os.getcwd()
for i in city_data.keys():
    print(i)
    # Load the data and preprocess
    data = combine_weather_data(city_data[i], cwd, date)
    data['Date'] = pd.to_datetime(data['Date'])
    
    # Extract features
    X = data['Date'].dt.dayofyear.values.reshape(-1, 1)  # Convert date to day of the year
    y = data['Temp'].values

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the Random Forest Regressor
    model = GradientBoostingRegressor(n_estimators=100, random_state=42)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    test_predictions = model.predict(X_test)

    # Compute and print the Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, test_predictions)
    print("Test MSE:", mse)

    # Make prediction for 2024-03-12
    date_to_predict = pd.to_datetime("2024-03-25").dayofyear
    predicted_temperature = model.predict([[date_to_predict]])
    print(f"Predicted temperature for {date}:", predicted_temperature[0])

    print(f'Removing forecast for {date}:')

    # Convert the "Date" column to datetime format
    # data['Date'] = pd.to_datetime(data['Date'])
    data = data[data['Date'].dt.date != pd.to_datetime(date).date()] #

    # Extract features
    X = data['Date'].dt.dayofyear.values.reshape(-1, 1)  # Convert date to day of the year
    y = data['Temp'].values

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the Random Forest Regressor
    model = GradientBoostingRegressor(n_estimators=100, random_state=42)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    test_predictions = model.predict(X_test)

    # Compute and print the Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, test_predictions)
    print("Test MSE:", mse)

    # Make prediction for 2024-03-12
    date_to_predict = pd.to_datetime(date).dayofyear
    predicted_temperature = model.predict([[date_to_predict]])
    print(f"Predicted temperature for {date}:", predicted_temperature[0])
    print()



New York
Test MSE: 64.17686068833737
Predicted temperature for 2024.03.25: 54.681554279247464
Removing forecast for 2024.03.25:
Test MSE: 62.850891005037006
Predicted temperature for 2024.03.25: 54.781336393937465

Austin
Test MSE: 83.6521873215173
Predicted temperature for 2024.03.25: 76.46293683622841
Removing forecast for 2024.03.25:
Test MSE: 81.76497269110828
Predicted temperature for 2024.03.25: 76.45952152720974

Chicago
Test MSE: 91.53435675673454
Predicted temperature for 2024.03.25: 49.69599763766677
Removing forecast for 2024.03.25:
Test MSE: 87.31214007044468
Predicted temperature for 2024.03.25: 49.26409672161121

Miami
Test MSE: 15.000087075853232
Predicted temperature for 2024.03.25: 82.99139126115224
Removing forecast for 2024.03.25:
Test MSE: 15.32068441631767
Predicted temperature for 2024.03.25: 83.2665567236451



### Gradient Boosting
#### Predicting on year, month, day as separate features

In [30]:
# Load the data
date = '2024.03.25'
cwd = os.getcwd()
for i in city_data.keys():
    print(i)
    # Load the data and preprocess
    data = combine_weather_data(city_data[i], cwd, date)
    data['Date'] = pd.to_datetime(data['Date'])
    
    # Separate the date column into year, month, and day columns
    data['Year'] = pd.to_datetime(data['Date']).dt.year
    data['Month'] = pd.to_datetime(data['Date']).dt.month
    data['Day'] = pd.to_datetime(data['Date']).dt.day

    # Extract features and split into training and test sets.
    X = data[['Year', 'Month', 'Day']]
    y = data['Temp'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train Gradient Boosted Tree Regressor.
    gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    gb_model.fit(X_train, y_train)

    # Make predictions on the test set and compute MSE.
    gb_test_predictions = gb_model.predict(X_test)
    gb_mse = mean_squared_error(y_test, gb_test_predictions)
    print("Gradient Boosted Test MSE:", round(gb_mse, 2))

    # Make prediction for 2024-03-12
    dt = datetime.strptime(date, '%Y.%m.%d') 
    date_to_predict = [[dt.year, dt.month, dt.day]]
    predicted_temperature = gb_model.predict(date_to_predict)
    print(f"Predicted temperature for {date}:", predicted_temperature[0])

    print(f'Removing forecast for {date}:')

    # Convert the "Date" column to datetime format
    # data['Date'] = pd.to_datetime(data['Date'])
    data = data[data['Date'].dt.date != pd.to_datetime(date).date()]

    # Separate the date column into year, month, and day columns
    data['Year'] = pd.to_datetime(data['Date']).dt.year
    data['Month'] = pd.to_datetime(data['Date']).dt.month
    data['Day'] = pd.to_datetime(data['Date']).dt.day

    # Extract features and split into training and test sets.
    X = data[['Year', 'Month', 'Day']]
    y = data['Temp'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train Gradient Boosted Tree Regressor.
    gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    gb_model.fit(X_train, y_train)

    # Make predictions on the test set and compute MSE.
    gb_test_predictions = gb_model.predict(X_test)
    gb_mse = mean_squared_error(y_test, gb_test_predictions)
    print("Gradient Boosted Test MSE:", round(gb_mse, 2))

    # Make prediction for 2024-03-12
    dt = datetime.strptime(date, '%Y.%m.%d') 
    date_to_predict = [[dt.year, dt.month, dt.day]]
    predicted_temperature = gb_model.predict(date_to_predict)
    print(f"Predicted temperature for {date}:", predicted_temperature[0])
    print()



New York
Gradient Boosted Test MSE: 60.09
Predicted temperature for 2024.03.25: 54.334188709650306
Removing forecast for 2024.03.25:


/opt/homebrew/Caskroom/miniconda/base/envs/cs542_test/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/cs542_test/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/cs542_test/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


Gradient Boosted Test MSE: 57.84
Predicted temperature for 2024.03.25: 54.71128010267876

Austin
Gradient Boosted Test MSE: 75.31
Predicted temperature for 2024.03.25: 77.59234618318064
Removing forecast for 2024.03.25:
Gradient Boosted Test MSE: 75.82
Predicted temperature for 2024.03.25: 78.88147161009552

Chicago


/opt/homebrew/Caskroom/miniconda/base/envs/cs542_test/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/cs542_test/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/cs542_test/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


Gradient Boosted Test MSE: 81.68
Predicted temperature for 2024.03.25: 55.98398281564241
Removing forecast for 2024.03.25:
Gradient Boosted Test MSE: 76.01
Predicted temperature for 2024.03.25: 51.25974759280538

Miami
Gradient Boosted Test MSE: 12.57
Predicted temperature for 2024.03.25: 81.21175775817592
Removing forecast for 2024.03.25:
Gradient Boosted Test MSE: 13.26
Predicted temperature for 2024.03.25: 81.72599983795998



/opt/homebrew/Caskroom/miniconda/base/envs/cs542_test/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/cs542_test/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
